Training CNN model based on 2 news channel

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# import libraries
import os
import cv2
import pandas as pd
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras import layers, models

In [ ]:
import os
import pandas as pd

# Set the paths to your "good" and "bad" parent folders
good_parent_folder = r'/content/drive/MyDrive/news/img_good'
bad_parent_folder = r'/content/drive/MyDrive/news/img_bad'

# Function to load images and label them
def load_images_and_label(folder_path, label):
    image_paths = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith(".jpg"):  # Adjust the file extension as needed
                image_paths.append(os.path.join(root, file))
    df = pd.DataFrame({'Image_Path': image_paths, 'Label': label})
    return df

# Load "good" images and label them as "good"
df_good = load_images_and_label(good_parent_folder, 'good')

# Load "bad" images and label them as "bad"
df_bad = load_images_and_label(bad_parent_folder, 'bad')

# Concatenate the dataframes
df = pd.concat([df_good, df_bad], ignore_index=True)

# Display the head of the dataframe
print(df.head())


Empty DataFrame
Columns: [Image_Path, Label]
Index: []


In [ ]:
df

,Image_Path,Label


In [ ]:
df.sample(2)

ValueError: ignored

In [ ]:
def preprocess_image(image_path, target_size=(224, 224)):
    # Read the image using OpenCV
    image = cv2.imread(image_path)

    # Resize the image to the target size
    image = cv2.resize(image, target_size)

    # Normalize pixel values to be in the range [0, 1]
    image = image.astype('float32') / 255.0

    return image

# Subsample the training data
df_train_subsampled = df.sample(n=1000, random_state=42)  # Adjust the sample size as needed

# Apply preprocessing to all images in the subsampled DataFrame
df_train_subsampled['Processed_Image'] = df_train_subsampled['Image_Path'].apply(preprocess_image)

# Convert labels to numerical format using LabelEncoder
label_encoder = LabelEncoder()
df_train_subsampled['Encoded_Label'] = label_encoder.fit_transform(df_train_subsampled['Label'])



ValueError: ignored

In [ ]:
df_train_subsampled

,Image_Path,Label,Processed_Image,Encoded_Label
989,/content/drive/MyDrive/news/img_good/video_1Q1...,good,"[[[0.7921569, 0.7372549, 0.6901961], [0.8, 0.7...",1
1636,/content/drive/MyDrive/news/img_bad/video_UKMb...,bad,"[[[0.44313726, 0.15294118, 0.019607844], [0.45...",0
1216,/content/drive/MyDrive/news/img_bad/video_5Wya...,bad,"[[[0.043137256, 0.043137256, 0.043137256], [0....",0
1174,/content/drive/MyDrive/news/img_good/video_dDF...,good,"[[[0.76862746, 0.6, 0.25882354], [0.7647059, 0...",1
1668,/content/drive/MyDrive/news/img_bad/video_zMzO...,bad,"[[[0.63529414, 0.39215687, 0.039215688], [0.63...",0
...,...,...,...,...
436,/content/drive/MyDrive/news/img_good/video_ZTt...,good,"[[[0.36862746, 0.32156864, 0.3137255], [0.3686...",1
770,/content/drive/MyDrive/news/img_good/video_-sd...,good,"[[[0.08627451, 0.16862746, 0.22745098], [0.101...",1
869,/content/drive/MyDrive/news/img_good/video_1NE...,good,"[[[0.4862745, 0.38039216, 0.3019608], [0.48627...",1
644,/content/drive/MyDrive/news/img_good/video_jDL...,good,"[[[0.7921569, 0.8039216, 0.79607844], [0.79215...",1


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Create an instance of the ImageDataGenerator
datagen = ImageDataGenerator(
    rescale=1./255,  # Rescale pixel values to [0, 1]
    rotation_range=40,  # Rotate images by 40 degrees
    width_shift_range=0.2,  # Shift images horizontally by 20%
    height_shift_range=0.2,  # Shift images vertically by 20%
    shear_range=0.2,  # Shear transformation
    zoom_range=0.2,  # Zoom in on images by 20%
    horizontal_flip=True,  # Flip images horizontally
    fill_mode='nearest'  # Fill in newly created pixels
)

# Create a generator for training data
train_generator = datagen.flow_from_dataframe(
    dataframe=df_train_subsampled,  # Your training dataframe
    x_col="Image_Path",  # Column containing image file paths
    y_col="Label",  # Column containing labels
    target_size=(150, 150),  # Resize images to 150x150
    batch_size=32,  # Batch size
    class_mode='binary'  # Binary classification
)

# Create a generator for validation data
# val_generator = datagen.flow_from_dataframe(...

Found 1000 validated image filenames belonging to 2 classes.


In [ ]:
# Update the build_and_compile_model function to handle binary classification correctly
def build_and_compile_model(input_shape=(224, 224, 3)):
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))

    # Use a sigmoid activation function and a single unit for binary classification
    model.add(layers.Dense(1, activation='sigmoid'))

    # Compile the model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    return model

# Example usage:
# Assuming you have two classes: 'good' and 'bad'
num_classes = 2
model = build_and_compile_model(input_shape=(224, 224, 3))

# Convert labels to numerical format using LabelEncoder
# label_encoder = LabelEncoder()
# df_train_subsampled['Encoded_Label'] = label_encoder.fit_transform(df_train_subsampled['Label'])

# Split the subsampled data into training and testing sets
X_train_subsampled, X_val_subsampled, y_train_subsampled, y_val_subsampled = train_test_split(
    np.array(df_train_subsampled['Processed_Image'].tolist()),
    df_train_subsampled['Encoded_Label'],
    test_size=0.2,
    random_state=42
)

# Shape of the training and validation sets
print(f"X_train_subsampled shape: {X_train_subsampled.shape}, y_train_subsampled shape: {y_train_subsampled.shape}")
print(f"X_val_subsampled shape: {X_val_subsampled.shape}, y_val_subsampled shape: {y_val_subsampled.shape}")

# Train your model
model.fit(X_train_subsampled, y_train_subsampled, epochs=20, batch_size=32, validation_data=(X_val_subsampled, y_val_subsampled))

X_train_subsampled shape: (800, 224, 224, 3), y_train_subsampled shape: (800,)
X_val_subsampled shape: (200, 224, 224, 3), y_val_subsampled shape: (200,)
Epoch 1/20
25/25 [==============================] - 13s 90ms/step - loss: 0.7189 - accuracy: 0.7262 - val_loss: 0.3481 - val_accuracy: 0.8200
Epoch 2/20
25/25 [==============================] - 1s 50ms/step - loss: 0.2618 - accuracy: 0.8938 - val_loss: 0.2836 - val_accuracy: 0.8900
Epoch 3/20
25/25 [==============================] - 1s 51ms/step - loss: 0.1427 - accuracy: 0.9450 - val_loss: 0.2502 - val_accuracy: 0.8900
Epoch 4/20
25/25 [==============================] - 1s 59ms/step - loss: 0.0706 - accuracy: 0.9775 - val_loss: 0.1510 - val_accuracy: 0.9600
Epoch 5/20
25/25 [==============================] - 1s 47ms/step - loss: 0.0333 - accuracy: 0.9962 - val_loss: 0.1682 - val_accuracy: 0.9450
Epoch 6/20
25/25 [==============================] - 1s 46ms/step - loss: 0.0166 - accuracy: 0.9950 - val_loss: 0.1706 - val_accuracy: 0.9350

In [ ]:
# Define the path to the new thumbnail dataset in Google Drive
new_data_folder = '/content/drive/MyDrive/news/Sky News'

# Load and preprocess images from the new dataset
def preprocess_image_for_prediction(image_path, target_size=(224, 224)):
    image = cv2.imread(image_path)
    image = cv2.resize(image, target_size)
    image = image.astype('float32') / 255.0
    return image

# List the images in the new dataset folder
new_data_images = [f for f in os.listdir(new_data_folder) if f.endswith(('.jpg', '.jpeg', '.png'))]

# Load the trained model
model = build_and_compile_model()  # Replace with the actual path to your trained model

# Make predictions on the new dataset
predictions = []

for image_file in new_data_images:
    image_path = os.path.join(new_data_folder, image_file)
    processed_image = preprocess_image_for_prediction(image_path)
    processed_image = np.expand_dims(processed_image, axis=0)  # Add batch dimension
    prediction = model.predict(processed_image)
    predictions.append({'image_file': image_file, 'prediction': prediction[0][0]})

# Display the predictions
for result in predictions:
    print(f"Image: {result['image_file']}, Prediction: {result['prediction']}")

1/1 [==============================] - 0s 28ms/step
Image: _J60fQr0GWo.jpg, Prediction: 0.6073764562606812
Image: yGPfKkjDIts.jpg, Prediction: 0.5017058849334717
Image: wZsaXxtfuRg.jpg, Prediction: 0.598988950252533
Image: u8cg5hhHJlA.jpg, Prediction: 0.6109470129013062
Image: swoAnLQ38NI.jpg, Prediction: 0.5376692414283752
Image: K8Z9Kqhrh5c.jpg, Prediction: 0.5828799605369568
Image: Mf76yyTY7Ss.jpg, Prediction: 0.626893937587738
Image: jr9XRmWNpfw.jpg, Prediction: 0.6068920493125916
Image: jG8Feqn6Z0E.jpg, Prediction: 0.5400630831718445
Image: N6cfFnBNELs.jpg, Prediction: 0.6288438439369202
Image: KH0SQbW-iaY.jpg, Prediction: 0.6450296640396118
Image: U4nBnuv9n9o.jpg, Prediction: 0.5507664680480957
Image: J1ugR-VVE1I.jpg, Prediction: 0.6203027367591858
Image: J-LqLU7CxeA.jpg, Prediction: 0.5649673342704773
Image: FnSr820S2Mk.jpg, Prediction: 0.5948818922042847
Image: hC4eog5cY9Q.jpg, Prediction: 0.636025607585907
Image: GESBI9pWWmI.jpg, Prediction: 0.5439183712005615
Image: FH-_Mtkug

In [ ]:
# Save the trained model
model.save('/content/drive/MyDrive/news/saved_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
import cv2
import os

# Load the saved model
saved_model_path = '/content/drive/MyDrive/news/saved_model.h5'
model = load_model(saved_model_path)

# Define the path to the new image
img_path = '/content/drive/MyDrive/news/Sky News/9E3FAKBlZh4.jpg'  # Replace with the actual path to your new image
img = image.load_img(img_path, target_size=(224, 224))  # Adjust the target size as needed
img = image.img_to_array(img)
img = np.expand_dims(img, axis=0)
img = img/255.0  # Normalize the image

# Make a prediction
result = model.predict(img)

# Check the prediction
if result[0][0] > 0.5:
    prediction = 'good'
    print('The image is labeled as good')
else:
    prediction = 'bad'
    print('The image is labeled as bad')


1/1 [==============================] - 0s 61ms/step
The image is labeled as good


#vgg 16 trial

In [ ]:
# Subsample size for each dataset
subsample_size = 1000

# Data augmentation and normalization
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)


good_images = []
good_labels = []

for filename in os.listdir(good_train_data_dir):
    img = cv2.imread(os.path.join(good_train_data_dir, filename))
    img = cv2.resize(img, (224, 224))
    img = img / 255.0  # Normalize pixel values
    good_images.append(img)
    good_labels.append(1)  # Assign label 1 for 'good'

# Load and preprocess data for 'bad' thumbnails
bad_images = []
bad_labels = []

for filename in os.listdir(bad_train_data_dir):
    img = cv2.imread(os.path.join(bad_train_data_dir, filename))
    img = cv2.resize(img, (224, 224))
    img = img / 255.0  # Normalize pixel values
    bad_images.append(img)
    bad_labels.append(0)  # Assign label 0 for 'bad'

# Convert lists to numpy arrays
good_images = np.array(good_images)
good_labels = np.array(good_labels)
bad_images = np.array(bad_images)
bad_labels = np.array(bad_labels)

# Combine the 'good' and 'bad' datasets
all_images = np.concatenate([good_images, bad_images])
all_labels = np.concatenate([good_labels, bad_labels])


X_train, X_val, y_train, y_val = train_test_split(
    all_images, all_labels, test_size=0.2, random_state=42)



In [ ]:
random_test_images = []
random_test_labels = []

for filename in os.listdir(random_test_data_dir):
    img = cv2.imread(os.path.join(random_test_data_dir, filename))
    img = cv2.resize(img, (224, 224))
    img = img / 255.0  # Normalize pixel values
    random_test_images.append(img)
    random_test_labels.append(0.5)  # Assign a label between 0 and 1 for 'random'

# Convert lists to numpy arrays
random_test_images = np.array(random_test_images)
random_test_labels = np.array(random_test_labels)

In [ ]:
# Define VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

model = models.Sequential()
model.add(base_model)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))


model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model on 'good' and 'bad' datasets
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_val, y_val))

# Evaluate the model on the random test dataset
test_loss, test_accuracy = model.evaluate(random_test_images, random_test_labels)
print(f'Test Loss: {test_loss}, Test Accuracy: {test_accuracy}')

58889256/58889256 [==============================] - 2s 0us/step
Epoch 1/10
43/43 [==============================] - 25s 205ms/step - loss: 3.7395 - accuracy: 0.6972 - val_loss: 0.5875 - val_accuracy: 0.7214
Epoch 2/10
43/43 [==============================] - 6s 135ms/step - loss: 0.5890 - accuracy: 0.7397 - val_loss: 0.5833 - val_accuracy: 0.7214
Epoch 3/10
43/43 [==============================] - 6s 135ms/step - loss: 0.5671 - accuracy: 0.7397 - val_loss: 0.5615 - val_accuracy: 0.7214
Epoch 4/10
43/43 [==============================] - 6s 136ms/step - loss: 0.5999 - accuracy: 0.7397 - val_loss: 0.5776 - val_accuracy: 0.7214
Epoch 5/10
43/43 [==============================] - 6s 136ms/step - loss: 0.5811 - accuracy: 0.7405 - val_loss: 0.5777 - val_accuracy: 0.7214
Epoch 6/10
43/43 [==============================] - 6s 135ms/step - loss: 0.5691 - accuracy: 0.7412 - val_loss: 0.5751 - val_accuracy: 0.7214
Epoch 7/10
43/43 [==============================] - 6s 136ms/step - loss: 0.5668 -